# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from math import *


In [2]:
#leitura do arquivo excel
budweiser=pd.read_excel("tweets_budweiser.xlsx", "Treinamento")
#budweiser=pd.read_excel("tweets_budweiser.xlsx", "Teste")

#print(budweiser)



In [3]:

#Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc.

def limpa_texto(ser_orig):
    ser = ser_orig.copy()
    ser=ser.str.replace('@', ' ')
    ser=ser.str.replace("'", ' ')
    ser=ser.str.replace('"', ' ')
    ser=ser.str.replace('.', ' ')
    ser=ser.str.replace('(', ' ')
    ser=ser.str.replace(')', ' ')
    ser=ser.str.replace('@', ' ')
    ser=ser.str.replace('$', ' ')
    ser=ser.str.replace('*', ' ')
    ser=ser.str.replace(':', ' ')
    ser=ser.str.replace(';', ' ')
    ser=ser.str.replace('-', ' ')
    ser=ser.str.replace('_', ' ')
    ser=ser.str.replace('%', ' ')
    ser=ser.str.replace('/', ' ')
    ser=ser.str.replace('!', ' ')
    ser=ser.str.replace('?', ' ')
    ser=ser.str.replace('[', ' ')
    ser=ser.str.replace(']', ' ')
    ser=ser.str.replace('{', ' ')
    ser=ser.str.replace('}', ' ')
    ser=ser.str.replace('+', ' ')
    ser=ser.str.replace('>', ' ')
    ser=ser.str.replace('<', ' ')
    ser=ser.str.replace('^', ' ')
    ser=ser.str.replace('~', ' ')  
    ser=ser.str.replace('=', ' ')
    ser=ser.str.replace('#', ' ')
    ser=ser.str.replace('+', ' ')
    ser=ser.str.replace('•', ' ')
    ser=ser.str.replace(',', ' ')


    #ser=ser.str.replace('...', ' ')

    #Propor outras limpezas/transformações que não afetem a qualidade da informação.
    ser=ser.str.replace('eu', ' ')
    ser=ser.str.replace('rt', ' ')
    ser=ser.str.replace('https://...', ' ')

    return ser
    
budweiser["Treinamento"] = limpa_texto(budweiser["Treinamento"])

print(budweiser)                                                                                                              

                                           Treinamento  Classificação
0     souto mc  anurbanshop  budweiser br  chinaski...              0
1    essa budweiser me fud  ontem  dor de cabeça tá...              1
2       david3 rick  crlh budweiser com red bull é ...              1
3       promodojaca  jogo do mengão amanhã   vem ve...              1
4       hailaffc  budweiser é a melhor cerveja e na...              1
5    1   welcome bud  quem for com camisa do eagles...              1
6       marianycostaa  budweiser é a melhor cerveja...              1
7     luizz alb prefiro budweiser  qua a e quinta e...              1
8       niallhbra2  📌| niall se apresenta hoje no b...              0
9    rolezinho q   dou valor  2 budweiser no mundia...              1
10   mais em compensação bebemos dois packs de budw...              1
11                não tem stella vai desce budweiser                1
12                  budweiser é a melhor cerveja   tnc              1
13   jogo do mengão 

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [4]:
conta_relevantes = len(budweiser[budweiser['Classificação'] == 1])
conta_relevantes

relevante=0
irrelevante=0
for i in budweiser['Classificação']:
    if i==0:
        irrelevante+=1
    elif i==1:
        relevante+=1
print(relevante)
print(irrelevante)
        
#Probabilidade de ser relevante

prob_relevante=relevante/300

prob_irrelevante=irrelevante/300
 
print(prob_relevante*100,prob_irrelevante*100)



194
106
64.66666666666666 35.333333333333336


In [5]:
relevantes_counts=budweiser["Treinamento"][budweiser.Classificação==1]
relevantes_counts

palavras_rel=[]
for r in relevantes_counts:
    for e in r.split():
        palavras_rel.append(e)
print(palavras_rel)

irrelevantes_counts=budweiser['Treinamento'][budweiser.Classificação==0]
palavras_irr=[]
for i in irrelevantes_counts:
    for e in i.split():
        palavras_irr.append(e)
print(palavras_irr)




['essa', 'budweiser', 'me', 'fud', 'ontem', 'dor', 'de', 'cabeça', 'tá', 'tendo', 'm', 's', 'amigos', '🤦🏿\u200d♂😂😂😂😂', 'david3', 'rick', 'crlh', 'budweiser', 'com', 'red', 'bull', 'é', 'um', 'caminho', 'sem', 'volta', 'deixa', 'mal', '\U0001f92d🍻🕺🏼', 'promodojaca', 'jogo', 'do', 'mengão', 'amanhã', 'vem', 'ver', 'aquele', 'jogo', 'tomando', 'uma', 'budweiser', 'gelada', 'mega', 'promoção', 'a', 'pa', 'i', 'de', 'amanhã', 'no', 'bar', 'do', 'leo', 'n…', 'hailaffc', 'budweiser', 'é', 'a', 'melhor', 'cerveja', 'e', 'nada', 'mais', 'impo', 'a', 'a', 'não', 'ser', 'minha', 'opinião', '1', 'welcome', 'bud', 'quem', 'for', 'com', 'camisa', 'do', 'eagles', 'ou', 'do', 'falcons', 'ganha', 'uma', 'budweiser', 'long', 'neck', '2', 'preço', 'da', 'pint', 'reduzido', 'pint', 'lager', 'schornstein', 'por', '11', '90', 'mais', 'barato', 'do', 'que', 'long', 'neck', '3', 'balde', 'promocional', 'balde', 'de', 'budweiser', 'compre', '5', 'leve', '6', 'a', 'noite', 'toda', 'marianycostaa', 'budweiser', 

In [6]:
dic_rel = {}
dic_ambas={}
for r in palavras_rel:
    dic_rel[r]=palavras_rel.count(r)
#print(dic_rel)

dic_irr = {}

for i in palavras_irr:
    dic_irr[i]=palavras_irr.count(i)
#print(dic_irr)

    

In [7]:
dic_prob_rel = {}


#print(dic_rel)

for r in dic_rel:
    dic_prob_rel[r] = (dic_rel[r] +1)/ ((len(dic_rel) + (len(dic_rel)+ len(dic_irr ))))
    
print(dic_prob_rel)

dic_prob_irr = {}

for i in dic_irr:
    dic_prob_irr[i] = (dic_irr[i] +1) / ((len(dic_irr) + (len(dic_rel)+ len(dic_irr ))))
    
print(dic_prob_irr)




{'essa': 0.0024580090126997134, 'budweiser': 0.08029496108152397, 'me': 0.005325686194182712, 'fud': 0.0012290045063498567, 'ontem': 0.0028676771814829987, 'dor': 0.0020483408439164277, 'de': 0.0340024580090127, 'cabeça': 0.0020483408439164277, 'tá': 0.004916018025399427, 'tendo': 0.0008193363375665711, 'm': 0.007374027038099139, 's': 0.0016386726751331422, 'amigos': 0.0008193363375665711, '🤦🏿\u200d♂😂😂😂😂': 0.0008193363375665711, 'david3': 0.0020483408439164277, 'rick': 0.0020483408439164277, 'crlh': 0.0028676771814829987, 'com': 0.009832036050798854, 'red': 0.0024580090126997134, 'bull': 0.0024580090126997134, 'é': 0.015157722244981565, 'um': 0.009832036050798854, 'caminho': 0.0024580090126997134, 'sem': 0.0036870135190495697, 'volta': 0.0028676771814829987, 'deixa': 0.0024580090126997134, 'mal': 0.004506349856616141, '\U0001f92d🍻🕺🏼': 0.0024580090126997134, 'promodojaca': 0.0020483408439164277, 'jogo': 0.004506349856616141, 'do': 0.01188037689471528, 'mengão': 0.0024580090126997134, 'a

In [8]:
#testes

bud_teste=pd.read_excel("tweets_budweiser.xlsx", "Teste")


bud_teste["Teste"] = limpa_texto(bud_teste["Teste"])



relevante_teste=0
irrelevante_teste=0
for i in bud_teste['Classificação']:
    if i==0:
        irrelevante_teste+=1
    elif i==1:
        relevante_teste+=1
print(relevante_teste)
print(irrelevante_teste)
        
#Probabilidade de ser relevante

prob_relevante_teste=relevante_teste/200

prob_irrelevante_teste=irrelevante_teste/200
 
print(prob_relevante_teste*100,prob_irrelevante_teste*100)



dic_teste = {}
dic_teste = bud_teste

relevantes_counts_teste=bud_teste["Teste"][bud_teste.Classificação==1]
relevantes_counts_teste=bud_teste["Teste"][bud_teste.Classificação==0]

palavras_rel_teste=[]
palavras_irr_teste=[]

for r in relevantes_counts_teste:
    for e in r.split():
        palavras_rel_teste.append(e)
        
for i in relevantes_counts_teste:
    for e in i.split():
        palavras_irr_teste.append(e)

#print(palavras_rel_teste)
#print(palavras_irre_teste)



dic_rel_teste = {}
for r in palavras_rel_teste:
    dic_rel_teste[r]=palavras_rel_teste.count(r)
print(dic_rel)

dic_irr_teste = {}

for i in palavras_irr_teste:
    dic_irr_teste[i]=palavras_irr_teste.count(i)
print(dic_irr)



dic_prob_rel_teste = {}


#print(dic_rel)

for r in dic_rel_teste:
    dic_prob_rel_teste[r] = (dic_rel_teste[r] +1)/ ((len(dic_rel_teste) + (len(dic_rel_teste)+ len(dic_irr_teste ))))
    
print(dic_prob_rel_teste)

dic_prob_irr_teste = {}

for i in dic_irr_teste:
    dic_prob_irr_teste[i] = (dic_irr_teste[i] +1) / ((len(dic_irr_teste) + (len(dic_rel_teste)+ len(dic_irr_teste ))))
    
print(dic_prob_irr_teste)










140
60
70.0 30.0
{'essa': 5, 'budweiser': 195, 'me': 12, 'fud': 2, 'ontem': 6, 'dor': 4, 'de': 82, 'cabeça': 4, 'tá': 11, 'tendo': 1, 'm': 17, 's': 3, 'amigos': 1, '🤦🏿\u200d♂😂😂😂😂': 1, 'david3': 4, 'rick': 4, 'crlh': 6, 'com': 23, 'red': 5, 'bull': 5, 'é': 36, 'um': 23, 'caminho': 5, 'sem': 8, 'volta': 6, 'deixa': 5, 'mal': 10, '\U0001f92d🍻🕺🏼': 5, 'promodojaca': 4, 'jogo': 10, 'do': 28, 'mengão': 5, 'amanhã': 11, 'vem': 5, 'ver': 5, 'aquele': 6, 'tomando': 14, 'uma': 66, 'gelada': 8, 'mega': 5, 'promoção': 7, 'a': 65, 'pa': 5, 'i': 5, 'no': 24, 'bar': 5, 'leo': 5, 'n…': 4, 'hailaffc': 1, 'melhor': 10, 'cerveja': 11, 'e': 72, 'nada': 2, 'mais': 15, 'impo': 2, 'não': 29, 'ser': 5, 'minha': 12, 'opinião': 2, '1': 6, 'welcome': 1, 'bud': 1, 'quem': 2, 'for': 2, 'camisa': 1, 'eagles': 1, 'ou': 4, 'falcons': 1, 'ganha': 1, 'long': 3, 'neck': 3, '2': 11, 'preço': 3, 'da': 21, 'pint': 2, 'reduzido': 1, 'lager': 1, 'schornstein': 1, 'por': 11, '11': 3, '90': 1, 'barato': 2, 'que': 41, '3': 3, 'b

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
